In [1]:
!pip install gcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602927 sha256=02bed7b31e49f745de28337d4893b5e1034af3d59d0d72e90b3900c3c8007b1a
  Stored in directory: /root/.cache/pip/wheels/7c/30/88/5017af921da3a33af785f0d0fd3e944b845bc62a445a2c2f69
Successfully built gcloud


In [2]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=WqcfpMYQe9M3s39fN1aKafeXQFGVoM&prompt=consent&token_usage=remote&access_type=offline&code_challenge=xX8A0EorGsSQ8dGubbCD1cMKjvDeUucXuOpBUWRvbvM&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AcvDMrCEMkt__kD8NTzr30P6c_EFw4_LjQTw4_8M0uc0pnOQY-in6M72CSdKzWUV6Z9P8Q

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [23]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re

In [24]:
df = pd.read_csv("ILOSTAT_genero.csv", usecols=['ref_area.label','source.label','time','sex.label','classif2.label','obs_value'])

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596 entries, 0 to 595
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ref_area.label  596 non-null    object 
 1   source.label    596 non-null    object 
 2   sex.label       596 non-null    object 
 3   classif2.label  596 non-null    object 
 4   time            596 non-null    int64  
 5   obs_value       596 non-null    float64
dtypes: float64(1), int64(1), object(4)
memory usage: 28.1+ KB


In [26]:
df.rename(columns={
    'ref_area.label': 'Reference area',
    'source.label': 'Source',
    'sex.label': 'Sex',
    'classif2.label': 'Setor',
    'time': 'Time',
    'obs_value': 'Total'
}, inplace=True)

In [27]:
df['Sex'] = df['Sex'].str.replace('Sex: ', '')
df['Setor'] = df['Setor'].str.replace('Institutional sector: ', '')
print(df)

    Reference area                                   Source     Sex    Setor  \
0        Argentina  LFS - Permanent Household Survey, Urban    Male   Public   
1        Argentina  LFS - Permanent Household Survey, Urban    Male  Private   
2        Argentina  LFS - Permanent Household Survey, Urban  Female   Public   
3        Argentina  LFS - Permanent Household Survey, Urban  Female  Private   
4        Argentina  LFS - Permanent Household Survey, Urban    Male   Public   
..             ...                                      ...     ...      ...   
591   South Africa      LFS - Quarterly Labour Force Survey  Female  Private   
592   South Africa      LFS - Quarterly Labour Force Survey    Male   Public   
593   South Africa      LFS - Quarterly Labour Force Survey    Male  Private   
594   South Africa      LFS - Quarterly Labour Force Survey  Female   Public   
595   South Africa      LFS - Quarterly Labour Force Survey  Female  Private   

     Time     Total  
0    2023  1079.5

In [28]:
df.head()

,Reference area,Source,Sex,Setor,Time,Total
0,Argentina,"LFS - Permanent Household Survey, Urban",Male,Public,2023,1079.515
1,Argentina,"LFS - Permanent Household Survey, Urban",Male,Private,2023,4265.218
2,Argentina,"LFS - Permanent Household Survey, Urban",Female,Public,2023,1332.894
3,Argentina,"LFS - Permanent Household Survey, Urban",Female,Private,2023,3192.356
4,Argentina,"LFS - Permanent Household Survey, Urban",Male,Public,2022,1049.845


In [29]:
df = df.rename(columns={'Time':'ano_pesquisa','Reference area':'pais','Source':'fonte_pesquisa','Total':'quantidade','Sex':'genero','Setor':'setor'})

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596 entries, 0 to 595
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pais            596 non-null    object 
 1   fonte_pesquisa  596 non-null    object 
 2   genero          596 non-null    object 
 3   setor           596 non-null    object 
 4   ano_pesquisa    596 non-null    int64  
 5   quantidade      596 non-null    float64
dtypes: float64(1), int64(1), object(4)
memory usage: 28.1+ KB


In [31]:
min(df['ano_pesquisa'])

2010

In [32]:
max(df['ano_pesquisa'])

2023

In [33]:
unique_countries = df['setor'].unique()
print(unique_countries)

translations = {
    'Private': 'Privado',
    'Public': 'Público'
}

# Substituindo os valores da coluna 'country' pelos valores traduzidos
df['setor'] = df['setor'].replace(translations)

print(df)

['Public' 'Private']
             pais                           fonte_pesquisa  genero    setor  \
0       Argentina  LFS - Permanent Household Survey, Urban    Male  Público   
1       Argentina  LFS - Permanent Household Survey, Urban    Male  Privado   
2       Argentina  LFS - Permanent Household Survey, Urban  Female  Público   
3       Argentina  LFS - Permanent Household Survey, Urban  Female  Privado   
4       Argentina  LFS - Permanent Household Survey, Urban    Male  Público   
..            ...                                      ...     ...      ...   
591  South Africa      LFS - Quarterly Labour Force Survey  Female  Privado   
592  South Africa      LFS - Quarterly Labour Force Survey    Male  Público   
593  South Africa      LFS - Quarterly Labour Force Survey    Male  Privado   
594  South Africa      LFS - Quarterly Labour Force Survey  Female  Público   
595  South Africa      LFS - Quarterly Labour Force Survey  Female  Privado   

     ano_pesquisa  quantidade 

In [34]:
unique_countries = df['genero'].unique()
print(unique_countries)

translations = {
    'Male': 'Homem',
    'Female': 'Mulher'
}

# Substituindo os valores da coluna 'country' pelos valores traduzidos
df['genero'] = df['genero'].replace(translations)

print(df)

['Male' 'Female']
             pais                           fonte_pesquisa  genero    setor  \
0       Argentina  LFS - Permanent Household Survey, Urban   Homem  Público   
1       Argentina  LFS - Permanent Household Survey, Urban   Homem  Privado   
2       Argentina  LFS - Permanent Household Survey, Urban  Mulher  Público   
3       Argentina  LFS - Permanent Household Survey, Urban  Mulher  Privado   
4       Argentina  LFS - Permanent Household Survey, Urban   Homem  Público   
..            ...                                      ...     ...      ...   
591  South Africa      LFS - Quarterly Labour Force Survey  Mulher  Privado   
592  South Africa      LFS - Quarterly Labour Force Survey   Homem  Público   
593  South Africa      LFS - Quarterly Labour Force Survey   Homem  Privado   
594  South Africa      LFS - Quarterly Labour Force Survey  Mulher  Público   
595  South Africa      LFS - Quarterly Labour Force Survey  Mulher  Privado   

     ano_pesquisa  quantidade  
0

In [35]:
df=df.rename(columns={'País':'pais'})

In [36]:
unique_countries = df['pais'].unique()
print(unique_countries)

translations = {
    'South Africa': 'África do Sul',
    'Argentina': 'Argentina',
    'Brazil': 'Brasil',
    'Chile': 'Chile',
    'Colombia': 'Colômbia',
    'United States of America': 'Estados Unidos',
    'France': 'França',
    'Mexico': 'México',
    'Peru': 'Peru',
    'Bolivia (Plurinational State of)': 'Bolívia',
    'Uruguay': 'Uruguai',
    'France' : 'França'
}

# Substituindo os valores da coluna 'country' pelos valores traduzidos
df['pais'] = df['pais'].replace(translations)

print(df)

['Argentina' 'Bolivia (Plurinational State of)' 'Brazil' 'Chile'
 'Colombia' 'France' 'Mexico' 'Peru' 'Uruguay' 'United States of America'
 'South Africa']
              pais                           fonte_pesquisa  genero    setor  \
0        Argentina  LFS - Permanent Household Survey, Urban   Homem  Público   
1        Argentina  LFS - Permanent Household Survey, Urban   Homem  Privado   
2        Argentina  LFS - Permanent Household Survey, Urban  Mulher  Público   
3        Argentina  LFS - Permanent Household Survey, Urban  Mulher  Privado   
4        Argentina  LFS - Permanent Household Survey, Urban   Homem  Público   
..             ...                                      ...     ...      ...   
591  África do Sul      LFS - Quarterly Labour Force Survey  Mulher  Privado   
592  África do Sul      LFS - Quarterly Labour Force Survey   Homem  Público   
593  África do Sul      LFS - Quarterly Labour Force Survey   Homem  Privado   
594  África do Sul      LFS - Quarterly Labo

In [37]:
schema=[
 bigquery.SchemaField('pais','STRING',description='Nome do país selecionado'),
 bigquery.SchemaField('fonte_pesquisa','STRING',description='De qual pesquisa nacional foi extraído aquele dado.'),
bigquery.SchemaField('genero','STRING',description='Gênero da pessoa'),
bigquery.SchemaField('quantidade','Float',description='total de pessoas que trabalham'),
bigquery.SchemaField('setor','STRING',description='se o setor é público ou privado'),
 bigquery.SchemaField('ano_pesquisa','INTEGER',description='Ano de coleta da informação'),

 ]

In [38]:
client = bigquery.Client(project='repositoriodedadosgpsp')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [39]:
dataset_ref = client.dataset('perfil_remuneracao')

In [40]:
table_ref = dataset_ref.table('ILOSTAT_paises_selecionados_genero')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=5c2686df-c093-4e97-a411-9c014ad6a645>